# use another position
- initial_state2, 2nd last is flipped
- has 2 circuits instead of 3 circuits.  11/16/2022

In [1]:
# !pip install pennylane
# !pip install pennylane-qiskit

In [ ]:
# !pip install pennylane-lightning

In [ ]:
# !pip install pennylane-lightning[gpu]  # has erro

# 2nd last flipped

In [ ]:
import numpy as np
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as qml_np
import pennylane as qml

import math
import matplotlib.pyplot as plt
import datetime

In [ ]:
num_of_qubits = 1+3
N = 2**(num_of_qubits-2)
# initial_state_phi1 = [math.sqrt(1/N)]*(N-1) + [0]*N + [math.sqrt(1/N)] # 2**(num_of_qubits-1)
# initial_state_phi1 = [.5,.5,.5, 0,   0, 0, 0, .5,  ] # 2**(num_of_qubits-1)

initial_state2 = [1/math.sqrt(N)]*(N-2) + [0, 1/math.sqrt(N)] + [0]*(N-2) + [1/math.sqrt(N), 0] # 2**(num_qubits-1)
initial_state_phi1 = initial_state2
print(f'initial_state_phi1={initial_state_phi1}')
# print(f'initial_state_phi1={initial_state_phi1[-5:]}')
initial_state_0_phi1  = initial_state_phi1 + [0]*len(initial_state_phi1) # 2**num_of_qubits


# print(f'initial_state3={initial_state3}')
# initial_state  = initial_state3 + [0]*len(initial_state3) # 2**num_qubits

initial_state_phi1=[0.5, 0.5, 0, 0.5, 0, 0, 0.5, 0]


In [ ]:
## three quantum circuits
def layer_with_HT(theta):
    for i in range(num_of_qubits-1):
        qml.CRY(theta[i], wires=(0,i+1))
    for i in range(num_of_qubits-2):
        qml.Toffoli(wires=(0,i+1,i+2))

        
def layer_no_HT(theta, qubit_posi):
    for i in range(num_of_qubits-1):
        qml.RY(theta[i], wires=(qubit_posi[i]))
    for i in range(num_of_qubits-2):
        qml.CNOT(wires=(qubit_posi[i],qubit_posi[i+1]))
    


In [ ]:
device_name = 'default.qubit'  #'default.qubit' # 
device_name2 = 'default.qubit' # has qml.state()

In [ ]:

        
# dev_with_HT=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HT=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HT)
def quantum_circuit_with_HT(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
#     qubit_position = list(range(1,num_of_qubits))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits)
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
    # return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
print(qml.draw(quantum_circuit_with_HT)([[0.1]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HT([[0.1]*(num_of_qubits-1)]))




0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│─────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X────┤     


In [ ]:

# dev_with_HTZ=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HTZ=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HTZ)
def quantum_circuit_with_HTZ(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
#     qubit_position = list(range(1,num_of_qubits))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits)
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.CZ([0,1])
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
    # return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
print('newly added')
print(qml.draw(quantum_circuit_with_HTZ)([[0.1]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HTZ([[0.1]*(num_of_qubits-1)]))



# dev_no_HT_Z=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, shots=20000, backend='qasm_simulator')
dev_no_HT_Z=qml.device(device_name, wires=num_of_qubits-1)        
@qml.qnode(dev_no_HT_Z)
def quantum_circuit_no_HT_return_Z(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))
    
    return qml.expval(qml.PauliZ(0))  
    # return qml.sample(qml.PauliZ(0)) 
print('newly added 2')   
print(qml.draw(quantum_circuit_no_HT_return_Z)([[0.2]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HT([[0.2]*(num_of_qubits-1)]))


# dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, backend='qasm_simulator')
dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits-1)  
@qml.qnode(dev_no_HT_S)
def quantum_circuit_no_HT_return_state(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))    
    return qml.state()

print('newly added 3')

newly added
0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│──╰Z────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●───────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X───────┤     
newly added 2
0: ─╭QubitStateVector(M0)──RY(0.20)─╭●────┤  <Z>
1: ─├QubitStateVector(M0)──RY(0.20)─╰X─╭●─┤     
2: ─╰QubitStateVector(M0)──RY(0.20)────╰X─┤     
newly added 3


In [ ]:
coef2 = 1
def objective_fn(theta):
    val1_1 = quantum_circuit_with_HT(theta)
    val1_2 = quantum_circuit_with_HTZ(theta)
#     val2 = quantum_circuit_no_HT_return_Z(theta)
#     return coef2*val2-0.5*(val1_1 - val1_2)
    return -0.5*(val1_1 - val1_2)

In [ ]:
max_repeat = 1 #100
iter_max = 300  #300
num_of_layers = 3
prb_last_list = []
obj_list_rep = []
theta_list = []
debug_flag, print_flag = 0, True
start_time = datetime.datetime.now()
for rep in range(1,max_repeat+1):
    if print_flag:
        print(f'\n\nrep={rep}')
    else:
        print(f'\n\nrep={rep}', end='  ')
    
    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta=qml_np.random.uniform(0, 2*math.pi, size=(num_of_layers, num_of_qubits-1), requires_grad=True)
    obj_list = []
    for iter in range(1, iter_max+1):    
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val2 = quantum_circuit_no_HT_return_Z(theta)
        if (iter==1 or iter%20==0 or iter==iter_max) and print_flag:
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} \
                 :: val1_2={val1_2:12.8f} :: val2={val2:12.8f}  \
                 coef2*val2-0.5*(val1_1 - val1_2)={coef2*val2-0.5*(val1_1 - val1_2):12.8f}  \
                 :: theta={theta}')
            # print(f'iter={iter:3d} :: obj={obj:12.8f} :: theta={theta}')
        obj_list.append(obj)
    theta_list.append(theta)
    print(f'obj_list(last 3)={obj_list[-3:]}')
    obj_list_rep.append(obj_list)
    
    ## display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
    prb = [i.item()**2 for i in state]
    # print(f'state={state}')
    if len(prb)>20:
        print(f'prb(last 2)={prb[-2:]}')
    else: print(f'prb={prb}')
    prb_last_list.append(prb[-1])
end_time = datetime.datetime.now()
duration = end_time - start_time
duration_in_s = duration.total_seconds()
print(f'time consumed: {duration_in_s}s')



rep=1
iter=  1 :: obj=  0.06869745 :: val1_1=  0.11519903                  :: val1_2=  0.18772902 :: val2=  0.55304731                   coef2*val2-0.5*(val1_1 - val1_2)=  0.58931231                   :: theta=[[0.08527298 5.45654854 3.26235483]
 [5.74774341 3.81395492 3.5094945 ]
 [0.14484543 0.15277612 5.80793008]]
iter= 20 :: obj= -0.36296344 :: val1_1=  0.59477002                  :: val1_2= -0.14429400 :: val2= -0.23469030                   coef2*val2-0.5*(val1_1 - val1_2)= -0.60422231                   :: theta=[[ 0.64545051  5.53967988  3.78105948]
 [ 6.61832943  4.74270508  4.33932407]
 [-0.80902031  0.4447367   6.64297412]]
iter= 40 :: obj= -0.45062139 :: val1_1=  0.55024144                  :: val1_2= -0.35443662 :: val2= -0.64746278                   coef2*val2-0.5*(val1_1 - val1_2)= -1.09980180                   :: theta=[[ 1.24653312  5.28025738  3.08560581]
 [ 6.87182318  5.03381248  4.06717977]
 [-1.45425391  0.15371302  6.44616459]]
iter= 60 :: obj= -0.47965128 :: val

In [ ]:
print('time now: ', end_time)

time now:  2023-03-13 20:33:50.713609


In [1]:
import numpy as np
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as qml_np
import pennylane as qml

import math
import matplotlib.pyplot as plt
import datetime


coef2 = 1
def objective_fn(theta):
    val1_1 = quantum_circuit_with_HT(theta)
    val1_2 = quantum_circuit_with_HTZ(theta)
#     val2 = quantum_circuit_no_HT_return_Z(theta)
#     return coef2*val2-0.5*(val1_1 - val1_2)
    return -0.5*(val1_1 - val1_2)

# 3rd last flipped

In [2]:

num_of_qubits = 1+3
N = 2**(num_of_qubits-2)
# initial_state_phi1 = [math.sqrt(1/N)]*(N-1) + [0]*N + [math.sqrt(1/N)] # 2**(num_of_qubits-1)
# initial_state_phi1 = [.5,.5,.5, 0,   0, 0, 0, .5,  ] # 2**(num_of_qubits-1)

initial_state3 = [1/math.sqrt(N)]*(N-3) + [0, 1/math.sqrt(N), 1/math.sqrt(N)] + [0]*(N-3) + [1/math.sqrt(N), 0, 0] # 2**(num_qubits-1)
initial_state_phi1 = initial_state3
# initial_state_phi1 = np.array([0.9, 0, 0.0, 0.0, 0, 0.1, 0, 0])/np.linalg.norm([0.9, 0, 0.0, 0.0, 0, 0.1, 0, 0])
initial_state_phi1 = [0.99388373 ,0.0,0.0, 0.0, 0.0, 0.11043153,0.0, 0.0 ]

# print(f'initial_state_phi1={initial_state_phi1} - {len(initial_state_phi1)}')
# initial_state_phi1 = [0.9, 0.0, 0.0, 0.0, 0.0, 0.1, 0, 0]/np.linalg.norm([0.9, 0, 0.0, 0.0, 0, 0.1, 0, 0])
# print(f'initial_state_phi1={initial_state_phi1} - {len(initial_state_phi1)}')

print(f'initial_state_phi1={initial_state_phi1}')
# print(f'initial_state_phi1={initial_state_phi1[-5:]}')
initial_state_0_phi1  = initial_state_phi1 + [0]*len(initial_state_phi1) # 2**num_of_qubits


# print(f'initial_state3={initial_state3}')
# initial_state  = initial_state3 + [0]*len(initial_state3) # 2**num_qubits

initial_state_phi1=[0.99388373, 0.0, 0.0, 0.0, 0.0, 0.11043153, 0.0, 0.0]


In [3]:
## three quantum circuits
def layer_with_HT(theta):
    for i in range(num_of_qubits-1):
        qml.CRY(theta[i], wires=(0,i+1))
    for i in range(num_of_qubits-2):
        qml.Toffoli(wires=(0,i+1,i+2))

        
def layer_no_HT(theta, qubit_posi):
    for i in range(num_of_qubits-1):
        qml.RY(theta[i], wires=(qubit_posi[i]))
    for i in range(num_of_qubits-2):
        qml.CNOT(wires=(qubit_posi[i],qubit_posi[i+1]))



In [4]:
device_name = 'default.qubit'  #'default.qubit' # 
device_name2 = 'default.qubit' # has qml.state()

In [5]:

        
# dev_with_HT=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HT=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HT)
def quantum_circuit_with_HT(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
#     qubit_position = list(range(1,num_of_qubits))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits)
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
    # return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
print(qml.draw(quantum_circuit_with_HT)([[0.1]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HT([[0.1]*(num_of_qubits-1)]))




0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│─────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X────┤     


In [6]:

# dev_with_HTZ=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HTZ=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HTZ)
def quantum_circuit_with_HTZ(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
#     qubit_position = list(range(1,num_of_qubits))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits)
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.CZ([0,1])
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
    # return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
print('newly added')
print(qml.draw(quantum_circuit_with_HTZ)([[0.1]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HTZ([[0.1]*(num_of_qubits-1)]))



# dev_no_HT_Z=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, shots=20000, backend='qasm_simulator')
dev_no_HT_Z=qml.device(device_name, wires=num_of_qubits-1)        
@qml.qnode(dev_no_HT_Z)
def quantum_circuit_no_HT_return_Z(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))
    
    return qml.expval(qml.PauliZ(0))  
    # return qml.sample(qml.PauliZ(0)) 
print('newly added 2')   
print(qml.draw(quantum_circuit_no_HT_return_Z)([[0.2]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HT([[0.2]*(num_of_qubits-1)]))


# dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, backend='qasm_simulator')
dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits-1)  
@qml.qnode(dev_no_HT_S)
def quantum_circuit_no_HT_return_state(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))    
    return qml.state()

print('newly added 3')

newly added
0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│──╰Z────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●───────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X───────┤     
newly added 2
0: ─╭QubitStateVector(M0)──RY(0.20)─╭●────┤  <Z>
1: ─├QubitStateVector(M0)──RY(0.20)─╰X─╭●─┤     
2: ─╰QubitStateVector(M0)──RY(0.20)────╰X─┤     
newly added 3


In [7]:
# coef2 = 1
# def objective_fn(theta):
#     val1_1 = quantum_circuit_with_HT(theta)
#     val1_2 = quantum_circuit_with_HTZ(theta)
#     val2 = quantum_circuit_no_HT_return_Z(theta)
#     return coef2*val2-0.5*(val1_1 - val1_2)

In [8]:
max_repeat = 1 #100
iter_max = 300  #300
num_of_layers = 3
prb_last_list = []
obj_list_rep = []
theta_list = []

test_prb=[]


debug_flag, print_flag = 0, True
start_time = datetime.datetime.now()
for rep in range(1,max_repeat+1):
    if print_flag:
        print(f'\n\nrep={rep}')
    else:
        print(f'\n\nrep={rep}', end='  ')
    
    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta=qml_np.random.uniform(0, 2*math.pi, size=(num_of_layers, num_of_qubits-1), requires_grad=True)
    obj_list = []
    for iter in range(1, iter_max+1):    
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val2 = quantum_circuit_no_HT_return_Z(theta)
        if (iter==1 or iter%40==0 or iter==iter_max) and print_flag:
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} \
                 :: val1_2={val1_2:12.8f} :: val2={val2:12.8f}  \
                 coef2*val2-0.5*(val1_1 - val1_2)={coef2*val2-0.5*(val1_1 - val1_2):12.8f}  \
                 :: theta={theta}')
            # print(f'iter={iter:3d} :: obj={obj:12.8f} :: theta={theta}')
        obj_list.append(obj)
    theta_list.append(theta)
    print(f'obj_list(last 3)={obj_list[-3:]}')
    obj_list_rep.append(obj_list)
    
    ## display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
    prb = [i.item()**2 for i in state]
    
    test_prb = prb
    
    # print(f'state={state}')
    if len(prb)>20:
        print(f'prb(last 3)={prb[-3:]}')
        print(f'prb(last 3)={prb[-1]}')
    else: print(f'prb={prb}')
    prb_last_list.append(prb[-3])
    
    
print(test_prb)

end_time = datetime.datetime.now()
duration = end_time - start_time
duration_in_s = duration.total_seconds()
print(f'time consumed: {duration_in_s}s')



rep=1
iter=  1 :: obj= -0.00989828 :: val1_1=  0.41404937                  :: val1_2=  0.37618980 :: val2=  0.23946681                   coef2*val2-0.5*(val1_1 - val1_2)=  0.22053703                   :: theta=[[3.90537033 1.34868762 1.55630995]
 [2.89190098 0.19381087 2.11840135]
 [4.42461288 2.1652948  5.67769758]]
iter= 40 :: obj= -0.10895325 :: val1_1=  0.09051358                  :: val1_2= -0.12741994 :: val2= -0.95783649                   coef2*val2-0.5*(val1_1 - val1_2)= -1.06680325                   :: theta=[[ 3.46384964  2.3604931   2.08079111]
 [ 2.97636391 -0.78192656  0.98230536]
 [ 2.98553578  2.76328162  6.57800322]]
iter= 80 :: obj= -0.10963231 :: val1_1=  0.06000516                  :: val1_2= -0.15927156 :: val2= -0.97144722                   coef2*val2-0.5*(val1_1 - val1_2)= -1.08108558                   :: theta=[[ 3.27319865  2.24984519  1.92779885]
 [ 2.99718678 -0.76282049  0.97749469]
 [ 3.1413783   2.84548487  6.63130836]]
iter=120 :: obj= -0.10980174 :: val

In [9]:
print('time now: ', end_time)

time now:  2023-03-13 20:50:11.475676
